# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene. You may define your own blender version.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.
* `optix_enable` : Enable OptiX which may boost performance, may be incompatible depending on the version of blender, project and GPU allocated

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more information.

In [1]:
blender_version = '4.5.3' #@param ['2.79b', '2.83.20', '2.93.18', '3.3.21', '3.6.23', '4.2.14', '4.5.3'] {allow-input: true}
blend_file_path = 'drive/MyDrive/untitled.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'google_drive' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'https://drive.google.com/file/d/1rDJ7EOD_BMj7c_qVk3JpSgua25PKYhY3/view?usp=drive_link' #@param {type: 'string'}
url_blend = 'https://drive.google.com/drive/folders/1pxlttsjTPdUq9inll39JrobIpqxtgW1M?usp=sharing' #@param {type: 'string'}
#@markdown ---
animation = True #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
download_type = 'gdrive_direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [2]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

/content
Current GPU: Tesla T4


In [3]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  google-perftools libgoogle-perftools4 libtcmalloc-minimal4
0 upgraded, 0 newly installed, 3 to remove and 38 not upgraded.
After this operation, 1,817 kB disk space will be freed.
(Reading database ... 126675 files and directories currently installed.)
Removing google-perftools (2.9.1-0ubuntu3) ...
Removing libgoogle-perftools4:amd64 (2.9.1-0ubuntu3) ...
Removing libtcmalloc-minimal4:amd64 (2.9.1-0ubuntu3) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtb

In [4]:
import shutil
import subprocess
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    if drive_path.startswith('https://drive.google.com/'):
        # Extract file ID from Google Drive URL
        file_id = drive_path.split('/')[-2]
        !gdown --id $file_id
        # Capture filename using subprocess and strip whitespace
        result = subprocess.run(['gdown', '--id', file_id, '--print-filename'], capture_output=True, text=True)
        uploaded_filename = result.stdout.strip()
    elif blend_file_path.startswith('https://drive.google.com/'):
        # Extract file ID from Google Drive URL
        file_id = blend_file_path.split('/')[-2]
        !gdown --id $file_id
        # Capture filename using subprocess and strip whitespace
        result = subprocess.run(['gdown', '--id', file_id, '--print-filename'], capture_output=True, text=True)
        uploaded_filename = result.stdout.strip()
    else:
        # Assume drive_path is a relative path within My Drive
        if not drive_path:
            raise SystemExit("Error: 'drive_path' is empty. Please provide the path to your file in Google Drive.")
        shutil.copy('/drive/MyDrive/' + drive_path, '.')
        uploaded_filename = os.path.basename(drive_path)

Mounted at /drive
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1rDJ7EOD_BMj7c_qVk3JpSgua25PKYhY3
From (redirected): https://drive.google.com/uc?id=1rDJ7EOD_BMj7c_qVk3JpSgua25PKYhY3&confirm=t&uuid=dac79476-dd78-4df4-a1d8-80f8922e3cfe
To: /content/untitled.blend
100% 118M/118M [00:02<00:00, 46.0MB/s]


In [5]:
import os
import shutil
import subprocess
from google.colab import files, drive

!rm -r render
!mkdir render

# Check if uploaded_filename is empty and try to determine filename from the download path
if not uploaded_filename:
    # Assuming the downloaded file name from QRzNmiHN8Xr0 is available in the output
    # This is a fallback and might not be reliable in all cases
    downloaded_file_path = "/content/untitled.blend" # This should match the download path in QRzNmiHN8Xr0
    if os.path.exists(downloaded_file_path):
        uploaded_filename = downloaded_file_path
        print(f"Note: uploaded_filename was empty, but found downloaded file at: {uploaded_filename}")
    else:
        print("Could not determine uploaded_filename and downloaded file not found at expected path.")
        uploaded_filename = "" # Ensure it's an empty string if retrieval fails


print(f"Processing file: {uploaded_filename}")

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/MyDrive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    print(f"Debug: uploaded_filename value is '{uploaded_filename}'")
    raise SystemExit(f"Invalid file extension for '{uploaded_filename}'. Only .blend and .zip files are supported.")

rm: cannot remove 'render': No such file or directory
Note: uploaded_filename was empty, but found downloaded file at: /content/untitled.blend
Processing file: /content/untitled.blend


In [6]:
import requests
import shutil
blender_url_dict = {
    '2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
    '2.83.20' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz" # example
    # The link will be inferred automatically even though the version is not defined here, you may override this behaviour by defining a new version here
    # Add any custom Linux binaries, refer to https://ftp.nluug.nl/pub/graphics/blender/release or other sites that provide direct link download
}

if blender_version in blender_url_dict:
    blender_url = blender_url_dict[blender_version]
else:
    major_minor = ".".join(blender_version.split('.')[:2])
    blender_url = f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major_minor}/blender-{blender_version}-linux-x64.tar.xz"

try:
    response = requests.head(blender_url, allow_redirects=True, timeout=10)
    if response.status_code != 200:
        print(f"Download failed for version '{blender_version}'.")
        print("Error downloading: You may need to define the download archive manually above.")
    else:
        base_url = os.path.basename(blender_url)
        print(f"Download URL: {blender_url}")
        print(f"Base filename: {base_url}")
except Exception as e:
    print(f"Error checking URL: {e}")
    print("Error downloading: You may need to define the download archive manually above.")

# Remove the directory if it exists before creating it
if os.path.exists(blender_version):
    shutil.rmtree(blender_version)
!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

Download URL: https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.5/blender-4.5.3-linux-x64.tar.xz
Base filename: blender-4.5.3-linux-x64.tar.xz
--2025-10-21 14:38:34--  https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.5/blender-4.5.3-linux-x64.tar.xz
Resolving ftp.nluug.nl (ftp.nluug.nl)... 145.220.21.40, 2001:67c:6ec:221:145:220:21:40
Connecting to ftp.nluug.nl (ftp.nluug.nl)|145.220.21.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 377397328 (360M) [application/octet-stream]
Saving to: ‘blender-4.5.3-linux-x64.tar.xz’

blender-4.5.3-linux 100%[===================>] 359.91M  40.5MB/s    in 9.6s    

2025-10-21 14:38:44 (37.7 MB/s) - ‘blender-4.5.3-linux-x64.tar.xz’ saved [377397328/377397328]



In [7]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "try:\n"+\
    "    scene.cycles.device = 'GPU'\n"+\
    "    prefs = bpy.context.preferences\n"+\
    "    prefs.addons['cycles'].preferences.get_devices()\n"+\
    "    cprefs = prefs.addons['cycles'].preferences\n"+\
    "    print(cprefs)\n"+\
    "    found_gpu = False\n"+\
    "    for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "        try:\n"+\
    "            cprefs.compute_device_type = compute_device_type\n"+\
    "            print('Device found:',compute_device_type)\n"+\
    "            found_gpu = True\n"+\
    "            break\n"+\
    "        except TypeError:\n"+\
    "            pass\n"+\
    "    if found_gpu:\n"+\
    "        for device in cprefs.devices:\n"+\
    "            if not re.match('intel', device.name, re.I):\n"+\
    "                print('Activating',device)\n"+\
    "                device.use = "+str(gpu_enabled)+"\n"+\
    "            else:\n"+\
    "                device.use = "+str(cpu_enabled)+"\n"+\
    "    else:\n"+\
    "        print('No Cycles GPU device found. Checking for CPU.')\n"+\
    "        if "+str(cpu_enabled)+":\n"+\
    "            scene.cycles.device = 'CPU'\n"+\
    "            print('Using CPU for rendering.')\n"+\
    "        else:\n"+\
    "            print('CPU rendering is not enabled. No rendering device available.')\n"+\
    "except Exception as e:\n"+\
    "    print(f'Error setting up rendering device: {e}')\n"+\
    "    if "+str(cpu_enabled)+":\n"+\
    "        scene.cycles.device = 'CPU'\n"+\
    "        print('Using CPU for rendering as fallback.')\n"+\
    "    else:\n"+\
    "        print('CPU rendering is not enabled and an error occurred. No rendering device available.')\n"

with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (e.g. Tesla K80) is not supported and you need to switch back to CUDA.")
    renderer = "OPTIX"

In [8]:
%cd /content

!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = '/content/output/' + output_name
else:
    output_path = '/drive/MyDrive/' + drive_output_path + output_name

%cd /content/$blender_version

if animation:
    if start_frame == end_frame:
        !./blender -b '{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./blender -b '{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./blender -b '{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"

/content
rm: cannot remove 'output': No such file or directory
/content/4.5.3
Blender 4.5.3 LTS (hash 67807e1800cc built 2025-09-09 01:38:15)
Read blend: "/content/untitled.blend"
<bpy_struct, CyclesPreferences at 0x79dbbeec7808>
Device found: CUDA
Activating <bpy_struct, CyclesDeviceSettings("Tesla T4") at 0x79dbbef9a388>
Activating <bpy_struct, CyclesDeviceSettings("Tesla T4") at 0x79dbbef9a498>
Fra:1 Mem:122.68M (Peak 123.05M) | Time:00:00.15 | Mem:0.00M, Peak:0.00M | Scene.001, ViewLayer | Synchronizing object | 衬衫马甲西装裤子条纹领带欧版西装套装_obj
Fra:1 Mem:122.68M (Peak 123.05M) | Time:00:00.15 | Mem:0.00M, Peak:0.00M | Scene.001, ViewLayer | Synchronizing object | Plane.001
Fra:1 Mem:125.66M (Peak 125.66M) | Time:00:00.15 | Mem:0.00M, Peak:0.00M | Scene.001, ViewLayer | Synchronizing object | Area.002
Fra:1 Mem:125.66M (Peak 125.66M) | Time:00:00.15 | Mem:0.00M, Peak:0.00M | Scene.001, ViewLayer | Synchronizing object | Area.003
Fra:1 Mem:217.42M (Peak 232.61M) | Time:00:00.41 | Mem:0.00M, Pe

In [10]:
%cd /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/MyDrive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/MyDrive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/MyDrive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

/content


## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020-2022 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```